# FIB-SEM Registration (FFT)

http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_feature_homography/py_feature_homography.html

http://docs.opencv.org/3.1.0/da/df5/tutorial_py_sift_intro.html

https://www.learnopencv.com/image-alignment-ecc-in-opencv-c-python/

Maybe I should be doing everything in terms of a homography matrix, then everything is matrix multiply.

In [ ]:
%matplotlib inline
import numpy as np
# better image reader than np
from scipy.misc import imread, imsave
import matplotlib.pyplot as plt
# three different registration packages
# not dft based
import cv2
# dft based
from skimage.feature import register_translation as register_translation_base

In [ ]:
%load_ext autoreload
%autoreload 2
from imreg_dph import *

In [ ]:
# AffineTransform Tests
af1 = AffineTransform(translation=(1, 2))
af2 = AffineTransform(translation=(5, 3))
af3 = af1 @ af2
assert np.array_equal(af3.translation, (6, 5))
assert af3 == af2 @ af1

af1 = AffineTransform(rotation=2)
af2 = AffineTransform(rotation=1)
af3 = af1 @ af2
assert af3.rotation == 3
assert af3 == af2 @ af1

In [ ]:
# testing
from skimage.data import astronaut
from skimage.color import rgb2gray

In [ ]:
astro = rgb2gray(astronaut())[:,:]

In [ ]:
plt.matshow(astro)

In [ ]:
logpolar(astro)[1]

In [ ]:
plt.matshow(logpolar(astro)[0])

In [ ]:
plt.matshow(warp(astro, AffineTransform(rotation=np.pi/4)))

In [ ]:
test_af = AffineTransform(rotation=.02, translation=(9, -5), scale=(.95, .95))
astro2 = warp(astro, test_af)
plt.matshow(astro2)

In [ ]:
from dphplotting import mip, slice_plot
from dphutils import slice_maker

In [ ]:
af = similarity(astro, astro2)

In [ ]:
print(af)

In [ ]:
resid = (af @ test_af)
print(resid)

In [ ]:
plt.matshow(astro2)
im2 = warp(astro, af)
plt.matshow(im2)
plt.matshow(astro)
plt.matshow(im2 - astro)

In [ ]:
plt.matshow(astro)

In [ ]:
af

In [ ]:
def register_translation(im0, im1, upsample_factor=100):
    """Right now this uses the numpy fft implementation, we can speed it up by
    dropping in fftw if we need to"""
    shifts, error, phasediff = register_translation_base(im0, im1, upsample_factor)
    af = AffineTransform(translation=shifts)
    return af

# OpenCV

In [ ]:
import cv2

In [ ]:
def register_ECC(im0, im1, warp_mode = cv2.MOTION_AFFINE, number_of_iterations = 50, termination_eps = 1e-3):
    """
    # Specify the number of iterations.

    # Specify the threshold of the increment
    # in the correlation coefficient between two iterations;
    """
    # Find size of image1
    sz = im0.shape

    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    # Run the ECC algorithm. The results are stored in warp_matrix.
    cc, warp_matrix = cv2.findTransformECC (im0, im1, warp_matrix, warp_mode, criteria)
    
    return AffineTransform(matrix=np.vstack((warp_matrix, (0,0,1))))

In [ ]:
# Find size of image1
sz = astro.shape
 
# Define the motion model
warp_mode = cv2.MOTION_AFFINE
 
# Define 2x3 or 3x3 matrices and initialize the matrix to identity
if warp_mode == cv2.MOTION_HOMOGRAPHY :
    warp_matrix = np.eye(3, 3, dtype=np.float32)
else :
    warp_matrix = np.eye(2, 3, dtype=np.float32)
    
# Specify the number of iterations.
number_of_iterations = 500;
 
# Specify the threshold of the increment
# in the correlation coefficient between two iterations
termination_eps = 1e-8;
 
# Define termination criteria
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)
 
# Run the ECC algorithm. The results are stored in warp_matrix.
%time (cc, warp_matrix) = cv2.findTransformECC (astro.astype(np.float32), astro2.astype(np.float32),warp_matrix, warp_mode, criteria)
print(warp_matrix)
if warp_mode == cv2.MOTION_HOMOGRAPHY :
    # Use warpPerspective for Homography 
    astro2_aligned = cv2.warpPerspective(astro2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
else :
    # Use warpAffine for Translation, Euclidean and Affine
    astro2_aligned = cv2.warpAffine(astro2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

In [ ]:
af2 = register_ECC(astro.astype(np.float32), astro2.astype(np.float32))
astro2_aligned = warp(astro2, af2)
# Show final results
plt.matshow(astro)
plt.matshow(astro2)
plt.matshow(astro2_aligned)
plt.matshow(astro2_aligned - astro)

In [ ]:
%timeit astro2_aligned = warp(astro2, af2)
%timeit astro2_aligned = cv2.warpAffine(astro2, af2.params[:2], (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

In [ ]:
resid = (af2 @ test_af)
print(resid.rotation)
print(resid.scale)
print(resid.shear)
print(resid.translation)